# Speculative Decoding

SGLang now provides an EAGLE-based (EAGLE-2/EAGLE-3) speculative decoding option. Our implementation aims to maximize speed and efficiency and is considered to be among the fastest in open-source LLM engines.
**Note:** Currently, Speculative Decoding in SGLang is compatible with radix cache and chunked prefill.

### Performance Highlights

Please see below for the huge improvements on throughput for LLaMA-Instruct 3.1 8B tested on MT bench that can be achieved via EAGLE3 decoding.
For further details please see the [EAGLE3 paper](https://arxiv.org/pdf/2503.01840).

| Method | Throughput (tokens/s) |
|--------|----------------|
| SGLang (w/o speculative, 1x H100) | 158.34 tokens/s |
| SGLang + EAGLE-2 (1x H100) | 244.10 tokens/s |
| SGLang + EAGLE-3 (1x H100) | 373.25 tokens/s |


## EAGLE Decoding

To enable EAGLE speculative decoding the following parameters are relevant:
* `speculative_draft_model_path`: Specifies draft model. This parameter is required.
* `speculative_num_steps`: Depth of autoregressive drafting. Increases speculation range but risks rejection cascades. Default is 5.

* `speculative_eagle_topk`: Branching factor per step. Improves candidate diversity, will lead to higher acceptance rate, but more lead to higher memory/compute consumption. Default is 4.

* `speculative_num_draft_tokens`: Maximum parallel verification capacity. Allows deeper tree evaluation but will lead to higher GPU memory usage. Default is 8.

These parameters are the same for EAGLE-2 and EAGLE-3.

### EAGLE-2 decoding

You can enable EAGLE-2 decoding by setting `--speculative_algorithm EAGLE` and choosing an appropriate model.

In [1]:
from sglang.test.test_utils import is_in_ci

if is_in_ci():
    from patch import launch_server_cmd
else:
    from sglang.utils import launch_server_cmd

from sglang.utils import wait_for_server, print_highlight, terminate_process

import openai

In [2]:
server_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model meta-llama/Llama-2-7b-chat-hf  --speculative-algorithm EAGLE \
    --speculative-draft-model-path lmsys/sglang-EAGLE-llama2-chat-7B --speculative-num-steps 3 \
    --speculative-eagle-topk 4 --speculative-num-draft-tokens 16 --cuda-graph-max-bs 8
"""
)

wait_for_server(f"http://localhost:{port}")

[2025-08-01 18:24:56] server_args=ServerArgs(model_path='meta-llama/Llama-2-7b-chat-hf', tokenizer_path='meta-llama/Llama-2-7b-chat-hf', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', model_loader_extra_config='{}', trust_remote_code=False, context_length=None, is_embedding=False, enable_multimodal=None, revision=None, model_impl='auto', host='127.0.0.1', port=34149, skip_server_warmup=False, warmups=None, nccl_port=None, dtype='auto', quantization=None, quantization_param_path=None, kv_cache_dtype='auto', mem_fraction_static=0.849, max_running_requests=200, max_queued_requests=9223372036854775807, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, hybrid_kvcache_ratio=None, swa_full_tokens_ratio=0.8, disable_hybrid_swa_memory=False, device='cuda', tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=33893

[2025-08-01 18:24:57] Using default HuggingFace chat template with detected content format: string


[2025-08-01 18:25:06] Attention backend not explicitly specified. Use flashinfer backend by default.
[2025-08-01 18:25:06] Init torch distributed begin.


[2025-08-01 18:25:06] Init torch distributed ends. mem usage=0.00 GB


[2025-08-01 18:25:07] Load weight begin. avail mem=62.07 GB


...

[2025-08-01 18:25:12] Using model weights format ['*.bin']
Loading pt checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


Loading pt checkpoint shards: 100% Completed | 1/1 [00:01<00:00,  1.23s/it]
Loading pt checkpoint shards: 100% Completed | 1/1 [00:01<00:00,  1.23s/it]

[2025-08-01 18:25:13] Load weight end. type=LlamaForCausalLMEagle, dtype=torch.float16, avail mem=54.23 GB, mem usage=0.93 GB.
[2025-08-01 18:25:13] KV Cache is allocated. #tokens: 20480, K size: 0.16 GB, V size: 0.16 GB
[2025-08-01 18:25:13] Memory pool end. avail mem=53.92 GB


[2025-08-01 18:25:13] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=200, context_len=4096, available_gpu_mem=54.14 GB
[2025-08-01 18:25:13] INFO:     Started server process [116090]
[2025-08-01 18:25:13] INFO:     Waiting for application startup.
[2025-08-01 18:25:13] INFO:     Application startup complete.
[2025-08-01 18:25:13] INFO:     Uvicorn running on http://127.0.0.1:34149 (Press CTRL+C to quit)


[2025-08-01 18:25:14] INFO:     127.0.0.1:41834 - "GET /v1/models HTTP/1.1" 200 OK


[2025-08-01 18:25:14] INFO:     127.0.0.1:41838 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-08-01 18:25:15] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 


In [3]:
client = openai.Client(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

response = client.chat.completions.create(
    model="meta-llama/Llama-2-7b-chat-hf",
    messages=[
        {"role": "user", "content": "List 3 countries and their capitals."},
    ],
    temperature=0,
    max_tokens=64,
)

print_highlight(f"Response: {response}")

[2025-08-01 18:26:21] Prefill batch. #new-seq: 1, #new-token: 16, #cached-token: 1, token usage: 0.00, #running-req: 1, #queue-req: 0, 


[2025-08-01 18:26:52] INFO:     127.0.0.1:41842 - "POST /generate HTTP/1.1" 200 OK
[2025-08-01 18:26:52] The server is fired up and ready to roll!


[2025-08-01 18:26:57] INFO:     127.0.0.1:45648 - "POST /v1/chat/completions HTTP/1.1" 200 OK


In [4]:
terminate_process(server_process)

### EAGLE-2 Decoding with `torch.compile`

You can also enable `torch.compile` for further optimizations and optionally set `--torch-compile-max-bs`:


In [5]:
server_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model meta-llama/Llama-2-7b-chat-hf  --speculative-algorithm EAGLE \
    --speculative-draft-model-path lmsys/sglang-EAGLE-llama2-chat-7B --speculative-num-steps 5 \
        --speculative-eagle-topk 8 --speculative-num-draft-tokens 64 --mem-fraction 0.6 \
            --enable-torch-compile --torch-compile-max-bs 2
"""
)

wait_for_server(f"http://localhost:{port}")

[2025-08-01 18:27:04] server_args=ServerArgs(model_path='meta-llama/Llama-2-7b-chat-hf', tokenizer_path='meta-llama/Llama-2-7b-chat-hf', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', model_loader_extra_config='{}', trust_remote_code=False, context_length=None, is_embedding=False, enable_multimodal=None, revision=None, model_impl='auto', host='127.0.0.1', port=33405, skip_server_warmup=False, warmups=None, nccl_port=None, dtype='auto', quantization=None, quantization_param_path=None, kv_cache_dtype='auto', mem_fraction_static=0.6, max_running_requests=200, max_queued_requests=9223372036854775807, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, hybrid_kvcache_ratio=None, swa_full_tokens_ratio=0.8, disable_hybrid_swa_memory=False, device='cuda', tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=8623223

[2025-08-01 18:27:04] Using default HuggingFace chat template with detected content format: string


[2025-08-01 18:27:14] Attention backend not explicitly specified. Use flashinfer backend by default.
[2025-08-01 18:27:14] Init torch distributed begin.


[2025-08-01 18:27:21] Init torch distributed ends. mem usage=1.47 GB


[2025-08-01 18:27:22] Load weight begin. avail mem=62.17 GB


...

Loading pt checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


Loading pt checkpoint shards: 100% Completed | 1/1 [00:01<00:00,  1.26s/it]
Loading pt checkpoint shards: 100% Completed | 1/1 [00:01<00:00,  1.26s/it]

[2025-08-01 18:27:29] Load weight end. type=LlamaForCausalLMEagle, dtype=torch.float16, avail mem=29.24 GB, mem usage=1.66 GB.
[2025-08-01 18:27:29] KV Cache is allocated. #tokens: 20480, K size: 0.16 GB, V size: 0.16 GB
[2025-08-01 18:27:29] Memory pool end. avail mem=28.93 GB


[2025-08-01 18:27:29] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=200, context_len=4096, available_gpu_mem=29.11 GB
[2025-08-01 18:27:29] INFO:     Started server process [123817]
[2025-08-01 18:27:29] INFO:     Waiting for application startup.
[2025-08-01 18:27:29] INFO:     Application startup complete.
[2025-08-01 18:27:29] INFO:     Uvicorn running on http://127.0.0.1:33405 (Press CTRL+C to quit)


[2025-08-01 18:27:30] INFO:     127.0.0.1:55160 - "GET /v1/models HTTP/1.1" 200 OK
[2025-08-01 18:27:30] INFO:     127.0.0.1:55164 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-08-01 18:27:30] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-08-01 18:27:33] INFO:     127.0.0.1:55174 - "POST /generate HTTP/1.1" 200 OK
[2025-08-01 18:27:33] The server is fired up and ready to roll!


In [6]:
client = openai.Client(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

response = client.chat.completions.create(
    model="meta-llama/Llama-2-7b-chat-hf",
    messages=[
        {"role": "user", "content": "List 3 countries and their capitals."},
    ],
    temperature=0,
    max_tokens=64,
)

print_highlight(f"Response: {response}")

[2025-08-01 18:27:35] Prefill batch. #new-seq: 1, #new-token: 16, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-08-01 18:27:36] INFO:     127.0.0.1:55190 - "POST /v1/chat/completions HTTP/1.1" 200 OK


In [7]:
terminate_process(server_process)

### EAGLE-2 Decoding via Frequency-Ranked Speculative Sampling

By employing a truncated high-frequency token vocabulary in the draft model, Eagle speculative decoding reduces `lm_head` computational overhead while accelerating the pipeline without quality degradation. For more details, checkout [the paper](https://arxiv.org/pdf/arXiv:2502.14856).

In our implementation, set `--speculative-token-map` to enable the optimization. You can get the high-frequency token in FR-Spec from [this model](https://huggingface.co/thunlp/LLaMA3-Instruct-8B-FR-Spec). Or you can obtain high-frequency token by directly downloading these token from [this repo](https://github.com/thunlp/FR-Spec/tree/main?tab=readme-ov-file#prepare-fr-spec-vocabulary-subset).

Thanks for the contribution from [Weilin Zhao](https://github.com/Achazwl) and [Zhousx](https://github.com/Zhou-sx). 

In [8]:
server_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model meta-llama/Meta-Llama-3-8B-Instruct --speculative-algorithm EAGLE \
    --speculative-draft-model-path lmsys/sglang-EAGLE-LLaMA3-Instruct-8B --speculative-num-steps 5 \
    --speculative-eagle-topk 8 --speculative-num-draft-tokens 64 --speculative-token-map thunlp/LLaMA3-Instruct-8B-FR-Spec/freq_32768.pt \
    --mem-fraction 0.7 --cuda-graph-max-bs 2 --dtype float16 
"""
)

wait_for_server(f"http://localhost:{port}")

[2025-08-01 18:27:43] server_args=ServerArgs(model_path='meta-llama/Meta-Llama-3-8B-Instruct', tokenizer_path='meta-llama/Meta-Llama-3-8B-Instruct', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', model_loader_extra_config='{}', trust_remote_code=False, context_length=None, is_embedding=False, enable_multimodal=None, revision=None, model_impl='auto', host='127.0.0.1', port=30658, skip_server_warmup=False, warmups=None, nccl_port=None, dtype='float16', quantization=None, quantization_param_path=None, kv_cache_dtype='auto', mem_fraction_static=0.7, max_running_requests=200, max_queued_requests=9223372036854775807, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, hybrid_kvcache_ratio=None, swa_full_tokens_ratio=0.8, disable_hybrid_swa_memory=False, device='cuda', tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, rand

[2025-08-01 18:27:44] Using default HuggingFace chat template with detected content format: string


[2025-08-01 18:27:52] Casting torch.bfloat16 to torch.float16.


[2025-08-01 18:27:52] Casting torch.bfloat16 to torch.float16.
[2025-08-01 18:27:52] Attention backend not explicitly specified. Use flashinfer backend by default.
[2025-08-01 18:27:52] Init torch distributed begin.


[2025-08-01 18:27:54] Init torch distributed ends. mem usage=0.00 GB


...

[2025-08-01 18:28:12] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=200, context_len=8192, available_gpu_mem=42.76 GB


[2025-08-01 18:28:13] INFO:     Started server process [126583]
[2025-08-01 18:28:13] INFO:     Waiting for application startup.
[2025-08-01 18:28:13] INFO:     Application startup complete.
[2025-08-01 18:28:13] INFO:     Uvicorn running on http://127.0.0.1:30658 (Press CTRL+C to quit)


[2025-08-01 18:28:13] INFO:     127.0.0.1:55398 - "GET /v1/models HTTP/1.1" 200 OK


[2025-08-01 18:28:14] INFO:     127.0.0.1:55414 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-08-01 18:28:14] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-08-01 18:28:16] INFO:     127.0.0.1:55418 - "POST /generate HTTP/1.1" 200 OK
[2025-08-01 18:28:16] The server is fired up and ready to roll!


In [9]:
client = openai.Client(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

response = client.chat.completions.create(
    model="meta-llama/Meta-Llama-3-8B-Instruct",
    messages=[
        {"role": "user", "content": "List 3 countries and their capitals."},
    ],
    temperature=0,
    max_tokens=64,
)

print_highlight(f"Response: {response}")

[2025-08-01 18:28:18] Prefill batch. #new-seq: 1, #new-token: 17, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-08-01 18:28:19] INFO:     127.0.0.1:55430 - "POST /v1/chat/completions HTTP/1.1" 200 OK


In [10]:
terminate_process(server_process)

### EAGLE-3 Decoding

You can enable EAGLE-3 decoding by setting `--speculative_algorithm EAGLE3` and choosing an appropriate model.

In [11]:
server_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model meta-llama/Llama-3.1-8B-Instruct  --speculative-algorithm EAGLE3 \
    --speculative-draft-model-path jamesliu1/sglang-EAGLE3-Llama-3.1-Instruct-8B --speculative-num-steps 5 \
        --speculative-eagle-topk 8 --speculative-num-draft-tokens 32 --mem-fraction 0.6 \
        --cuda-graph-max-bs 2 --dtype float16
"""
)

wait_for_server(f"http://localhost:{port}")

[2025-08-01 18:28:26] server_args=ServerArgs(model_path='meta-llama/Llama-3.1-8B-Instruct', tokenizer_path='meta-llama/Llama-3.1-8B-Instruct', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', model_loader_extra_config='{}', trust_remote_code=False, context_length=None, is_embedding=False, enable_multimodal=None, revision=None, model_impl='auto', host='127.0.0.1', port=39083, skip_server_warmup=False, warmups=None, nccl_port=None, dtype='float16', quantization=None, quantization_param_path=None, kv_cache_dtype='auto', mem_fraction_static=0.6, max_running_requests=200, max_queued_requests=9223372036854775807, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, hybrid_kvcache_ratio=None, swa_full_tokens_ratio=0.8, disable_hybrid_swa_memory=False, device='cuda', tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_see

[2025-08-01 18:28:26] Using default HuggingFace chat template with detected content format: string


[2025-08-01 18:28:34] Casting torch.bfloat16 to torch.float16.


[2025-08-01 18:28:35] Casting torch.bfloat16 to torch.float16.
[2025-08-01 18:28:35] Attention backend not explicitly specified. Use flashinfer backend by default.
[2025-08-01 18:28:35] Init torch distributed begin.


[2025-08-01 18:28:35] Init torch distributed ends. mem usage=0.00 GB


...

[2025-08-01 18:28:52] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=200, context_len=131072, available_gpu_mem=40.22 GB


[2025-08-01 18:28:53] INFO:     Started server process [128978]
[2025-08-01 18:28:53] INFO:     Waiting for application startup.
[2025-08-01 18:28:53] INFO:     Application startup complete.
[2025-08-01 18:28:53] INFO:     Uvicorn running on http://127.0.0.1:39083 (Press CTRL+C to quit)


[2025-08-01 18:28:53] INFO:     127.0.0.1:50418 - "GET /v1/models HTTP/1.1" 200 OK


[2025-08-01 18:28:54] INFO:     127.0.0.1:50420 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-08-01 18:28:54] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-08-01 18:28:57] INFO:     127.0.0.1:50430 - "POST /generate HTTP/1.1" 200 OK
[2025-08-01 18:28:57] The server is fired up and ready to roll!


In [12]:
client = openai.Client(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

response = client.chat.completions.create(
    model="meta-llama/Meta-Llama-3.1-8B-Instruct",
    messages=[
        {"role": "user", "content": "List 3 countries and their capitals."},
    ],
    temperature=0,
    max_tokens=64,
)

print_highlight(f"Response: {response}")

[2025-08-01 18:28:58] Prefill batch. #new-seq: 1, #new-token: 42, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-08-01 18:28:58] INFO:     127.0.0.1:50440 - "POST /v1/chat/completions HTTP/1.1" 200 OK


In [13]:
terminate_process(server_process)

## Multi Token Prediction

We support [MTP(Multi-Token Prediction)](https://arxiv.org/pdf/2404.19737) in SGLang by using speculative decoding. We use Xiaomi/MiMo-7B-RL model as example here (deepseek mtp usage refer to [deepseek doc](../references/deepseek.md#multi-token-prediction))

In [14]:
server_process, port = launch_server_cmd(
    """
    python3 -m sglang.launch_server --model-path XiaomiMiMo/MiMo-7B-RL --host 0.0.0.0 --trust-remote-code \
    --speculative-algorithm EAGLE --speculative-num-steps 1 --speculative-eagle-topk 1 --speculative-num-draft-tokens 2 \
    --mem-fraction 0.5
"""
)

wait_for_server(f"http://localhost:{port}")

[2025-08-01 18:29:07] server_args=ServerArgs(model_path='XiaomiMiMo/MiMo-7B-RL', tokenizer_path='XiaomiMiMo/MiMo-7B-RL', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', model_loader_extra_config='{}', trust_remote_code=True, context_length=None, is_embedding=False, enable_multimodal=None, revision=None, model_impl='auto', host='0.0.0.0', port=30253, skip_server_warmup=False, warmups=None, nccl_port=None, dtype='auto', quantization=None, quantization_param_path=None, kv_cache_dtype='auto', mem_fraction_static=0.5, max_running_requests=200, max_queued_requests=9223372036854775807, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, hybrid_kvcache_ratio=None, swa_full_tokens_ratio=0.8, disable_hybrid_swa_memory=False, device='cuda', tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=134555916, constrained_jso

[2025-08-01 18:29:08] Using default HuggingFace chat template with detected content format: string


[2025-08-01 18:29:15] Attention backend not explicitly specified. Use flashinfer backend by default.
[2025-08-01 18:29:15] Init torch distributed begin.


[2025-08-01 18:29:16] Init torch distributed ends. mem usage=0.00 GB


[2025-08-01 18:29:17] Load weight begin. avail mem=78.58 GB


...

[2025-08-01 18:29:22] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=200, context_len=32768, available_gpu_mem=59.84 GB


[2025-08-01 18:29:22] INFO:     Started server process [130979]
[2025-08-01 18:29:22] INFO:     Waiting for application startup.
[2025-08-01 18:29:22] INFO:     Application startup complete.
[2025-08-01 18:29:22] INFO:     Uvicorn running on http://0.0.0.0:30253 (Press CTRL+C to quit)


[2025-08-01 18:29:22] INFO:     127.0.0.1:52174 - "GET /v1/models HTTP/1.1" 200 OK


[2025-08-01 18:29:23] INFO:     127.0.0.1:52176 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-08-01 18:29:23] Prefill batch. #new-seq: 1, #new-token: 6, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-08-01 18:29:25] INFO:     127.0.0.1:52184 - "POST /generate HTTP/1.1" 200 OK
[2025-08-01 18:29:25] The server is fired up and ready to roll!


In [15]:
import requests

url = f"http://localhost:{port}/v1/chat/completions"

data = {
    "model": "XiaomiMiMo/MiMo-7B-RL",
    "messages": [{"role": "user", "content": "What is the capital of France?"}],
}

response = requests.post(url, json=data)
print_highlight(response.json())

[2025-08-01 18:29:27] Prefill batch. #new-seq: 1, #new-token: 26, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-08-01 18:29:28] Decode batch. #running-req: 1, #token: 91, token usage: 0.00, accept len: 1.76, cuda graph: False, gen throughput (token/s): 11.14, #queue-req: 0, 


[2025-08-01 18:29:29] Decode batch. #running-req: 1, #token: 157, token usage: 0.01, accept len: 1.65, cuda graph: False, gen throughput (token/s): 105.18, #queue-req: 0, 


[2025-08-01 18:29:29] Decode batch. #running-req: 1, #token: 224, token usage: 0.01, accept len: 1.68, cuda graph: False, gen throughput (token/s): 102.98, #queue-req: 0, 


[2025-08-01 18:29:30] Decode batch. #running-req: 1, #token: 286, token usage: 0.01, accept len: 1.55, cuda graph: False, gen throughput (token/s): 98.73, #queue-req: 0, 


...

[2025-08-01 18:29:31] Decode batch. #running-req: 1, #token: 421, token usage: 0.02, accept len: 1.75, cuda graph: False, gen throughput (token/s): 109.62, #queue-req: 0, 


[2025-08-01 18:29:32] Decode batch. #running-req: 1, #token: 487, token usage: 0.02, accept len: 1.65, cuda graph: False, gen throughput (token/s): 102.78, #queue-req: 0, 


[2025-08-01 18:29:33] Decode batch. #running-req: 1, #token: 556, token usage: 0.03, accept len: 1.73, cuda graph: False, gen throughput (token/s): 107.95, #queue-req: 0, 


[2025-08-01 18:29:33] Decode batch. #running-req: 1, #token: 624, token usage: 0.03, accept len: 1.70, cuda graph: False, gen throughput (token/s): 108.89, #queue-req: 0, 


[2025-08-01 18:29:33] INFO:     127.0.0.1:52186 - "POST /v1/chat/completions HTTP/1.1" 200 OK


In [16]:
terminate_process(server_process)

## References

EAGLE process is as follows:

- Within EAGLE the draft model predicts the next feature vector, i.e. the last hidden state of the original LLM, using the feature sequence $(f_1, ..., f_k)$ and the token sequence $(t_2, ..., t_{k+1})$. 
- The next token is then sampled from $p_{k+2}=\text{LMHead}(f_{k+1})$. Afterwards, the two sequences are extended in a tree style—branching out multiple potential continuations, with the branching factor per step controlled by the `speculative_eagle_topk` parameter—to ensure a more coherent connection of context, and are given as input again.
- EAGLE-2 additionally uses the draft model to evaluate how probable certain branches in the draft tree are, dynamically stopping the expansion of unlikely branches. After the expansion phase, reranking is employed to select only the top `speculative_num_draft_tokens` final nodes as draft tokens.
- EAGLE-3 removes the feature prediction objective, incorporates low and mid-layer features, and is trained in an on-policy manner.

This enhances drafting accuracy by operating on the features instead of tokens for more regular inputs and passing the tokens from the next timestep additionally to minimize randomness effects from sampling. Furthermore the dynamic adjustment of the draft tree and selection of reranked final nodes increases acceptance rate of draft tokens further. For more details see [EAGLE-2](https://arxiv.org/abs/2406.16858) and [EAGLE-3](https://arxiv.org/abs/2503.01840) paper.


For guidance how to train your own EAGLE model please see the [EAGLE repo](https://github.com/SafeAILab/EAGLE/tree/main?tab=readme-ov-file#train).